# Importing Dependencies

In [ ]:
%%capture
# Importing Basic Libs
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

#Yahoo Finance
!pip install yfinance
import yfinance as yf


# Macro Data & FRED API
!pip install fredapi
from fredapi import Fred
import plotly.express as px
api_key_fred = "3daf7224721fb969638ce10e4ac886fc"

#Regression Packages
!pip install scikit-learn
from sklearn.linear_model import LinearRegression
!pip install statsmodels
import seaborn as sns
sns.set()

#Portfolio Optimizer 
!pip install pyportfolioopt


# Importing Data Sets 
## Using Y-Finance to extract prices

*   Investment 1: Bond : '^FVX' US Treasury Bill 5 Years
*   Investment 2: Income Stock :  'SPHD' Invesco S&P 500 High Dividend Low Volatility
*   Investment 3: Cryptocurrency : Unable to download closing price data for BTC-USD from Yahoo Finance
*   Investment 4: Equity ETF : '^GSPC' S & P 500
*   Investment 5: Crypto ETF : 'BITO' ProShares Bitcoin Strategy ETF 
*   Investment 6: Stock :   ETF'AAPL' Apple







In [ ]:
ticker_list = ['^FVX' , 'SPHD' , '^GSPC', 'BITO', 'AAPL']

In [ ]:
# Extracting Static Price Data
def load_ticker(tickers):
    df_0 = yf.download(tickers, period = "max")
    return df_0

df = load_ticker(ticker_list)['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [ ]:
# Cleaning Data
df.dropna(inplace = True)
df.head()

,AAPL,BITO,SPHD,^FVX,^GSPC
Date,,,,,
2021-10-20 00:00:00-04:00,148.182465,43.279999,41.755596,1.147,4536.189941
2021-10-21 00:00:00-04:00,148.400864,40.830002,41.450531,1.212,4549.779785
2021-10-22 00:00:00-04:00,147.616577,39.509998,41.488663,1.211,4544.899902
2021-10-25 00:00:00-04:00,147.566925,40.580002,41.412399,1.176,4566.479980
2021-10-26 00:00:00-04:00,148.242035,40.060001,41.460072,1.183,4574.790039


In [ ]:
# Calculating Returns 
returns = df.pct_change(fill_method ='ffill')
returns.dropna(inplace = True)
returns.head()

,AAPL,BITO,SPHD,^FVX,^GSPC
Date,,,,,
2021-10-21 00:00:00-04:00,0.001474,-0.056608,-0.007306,0.056670,0.002996
2021-10-22 00:00:00-04:00,-0.005285,-0.032329,0.000920,-0.000825,-0.001073
2021-10-25 00:00:00-04:00,-0.000336,0.027082,-0.001838,-0.028902,0.004748
2021-10-26 00:00:00-04:00,0.004575,-0.012814,0.001151,0.005952,0.001820
2021-10-27 00:00:00-04:00,-0.003148,-0.049925,-0.010577,-0.038039,-0.005052


In [ ]:
def display_summary(returns):
  # Describing Data 
  print(returns.describe())

  # Calculating Mean
  returns_mean = returns.mean()
  print('Mean:')
  print(returns_mean)

  # Calcualting Variance 
  returns_var = returns.var()
  print('Var:')
  print(returns_var)

  # Calculating skewness 
  returns_skew = returns.skew()
  print('Skew :')
  print(returns_skew)

  # Calculating Kurtosis
  returns_kurtosis = returns.kurtosis()
  print('Kurtosis :')
  print(returns_kurtosis)

  # Calculating Correlation Matrix 
  returns_corr = returns.corr()
  print('Correlations :')
  print(returns_corr)

  # Calculating Covaraince Matrix 
  returns_cov = returns.cov()
  print('Covariance :')
  print(returns_corr)
  return returns_mean, returns_var, returns_skew, returns_kurtosis, returns_corr, returns_cov

# Portfolio -1 

Buy 1 Asset & Sell 1

Since we are creating a portfolio of two assets, we will create an asset that is short markets.

Hence we picked ^GSPC 50% and  ^FVX 50%, shoritng the ^GSPC

In [ ]:
returns['P1'] = -0.5 * returns['^GSPC'] + 0.5 * returns['^FVX']

# Portfolio - 2
Buy any number of assets

## Risk Effecient Frontier 
We can use a python package that calcualtes the optimal portfolio allocaiton based on mean varaince optimisaiton. The package uses the asset return charectersistics to maxsimise Sharpe Ratio

In [ ]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier

mu = mean_historical_return(df)
S = CovarianceShrinkage(df).ledoit_wolf()


ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(dict(cleaned_weights))

{'AAPL': 0.0, 'BITO': 0.0, 'SPHD': 0.2966, '^FVX': 0.7034, '^GSPC': 0.0}


In [ ]:
# Using the weights above we consturct a potfolio : 
returns['P2'] = 0.2966 * returns['SPHD'] + 0.0.7034 * returns['^FVX']

# Portfolio - 3
 
Buy and Sell multiple Assets, with constraint N>3.

For portoflio 3, use a uniform allocaiton for all assets invovled, with 20% each, whiilst shorting the bitcoin ETF

In [ ]:
returns['P3'] = returns['AAPL'] * 0.2 + returns['BITO'] * -0.2 + returns['SPHD'] * 0.2 + returns['^FVX'] * 0.2 +  returns['^GSPC'] * 0.2 

# Portfolio Summar Statistics


In [ ]:
portfolio = pd.DataFrame(list(zip(returns['P1'], returns['P2'], returns['P3'])), columns = ['P1', 'P2', 'P3'])
mean,var, skew, kurt, corr, cov = display_summary(portfolio)

             AAPL        BITO        SPHD        ^FVX       ^GSPC          P1  \
count  319.000000  319.000000  319.000000  319.000000  319.000000  319.000000   
mean     0.000179   -0.002554    0.000294    0.004159   -0.000239    0.002199   
std      0.021346    0.040625    0.010674    0.033265    0.014187    0.018477   
min     -0.058680   -0.202228   -0.036368   -0.130208   -0.043237   -0.067624   
25%     -0.013188   -0.023545   -0.006471   -0.015055   -0.008438   -0.009582   
50%      0.000000   -0.002676    0.000920    0.003482   -0.000830    0.002189   
75%      0.014732    0.019181    0.006634    0.022383    0.009332    0.012413   
max      0.088975    0.124224    0.039699    0.136704    0.055434    0.072465   

               P2          P3  
count  319.000000  319.000000  
mean     0.003061    0.001389  
std      0.023767    0.010288  
min     -0.097350   -0.037844  
25%     -0.009948   -0.005049  
50%      0.001943    0.002206  
75%      0.016509    0.007376  
max      0.099

# Portfolio Write up


1.   P1 : 50% Short S&P500 & 50% Long Bonds
  *   Contains credit risk, as it contains treassury bills.
  * Yes the protfolio can be short sold.Using inverse etfs, without having to replicate the portoflio by borriwing and selling is one way to do. The other way is to sell short the bond, and use its preimum and the risk free borrowing to finance the purchase of the long asset. 
  * Mean Daily Return 0.2% 
  *  Daily Variance = 0.0341%
  * The Portfolio does not contain good diversificatoin, as we are going long and short with negative corelation assets, which means the returns of the assets in the portfolio is moving together
  * P1 takes a very storng directional bet, and this makes it suspebitble to event driven scenarios
  * In periods of earnings expansio and expansionary monetary policy, the portfolio will perform well. 
  * Skew reduced. Both indidual elemnts have a skew close to 0.19, but the combined portfolio reduces skew to 0.17. 

2.   P2 : Long High Yeild ETF 30% & Long Bonds 70%
  *   Contains credit risk, as it contains treassury bills.
  * Yes the protfolio can be short sold.Using inverse etfs, without having to replicate the portoflio by borriwing and selling is one way to do. The other way is to sell short the bond, and use its preimum and the risk free borrowing to finance the purchase of the long asset. 
  * Mean Daily Return 0.3061% 
  *  Daily Variance = 0.0565%
  * The Portfolio does not contain good diversificatoin, as the returns are exremely volatile. The retunrs have increased, but so has the variance.
  * P2 takes a very storng hedging stratergy bet, and this makes it suspebitimmune ble to event driven scenarios, as it allows for stable earnings.
  * In periods of earnings expansio and expansionary monetary policy, the portfolio will not perform well, due to long exposure to the bond.  
  * Skew increased in comparison to both individual components.  


3. P3 : 20% weightage to all, Long all, and Short Bond
  *   Contains credit risk, as it contains treassury bills.
  * Yes the protfolio can be short sold.Using inverse etfs, without having to replicate the portoflio by borriwing and selling is one way to do. The other way is to sell short the bond, and use its preimum and the risk free borrowing to finance the purchase of the long asset. 
  * Mean Daily Return 0.1389% 
  *  Daily Variance = 0.0106%
  * The Portfolio is better diversified as the variance has considerably reduced. This is due to the fact that the porfolio is inevsted in more assets without a perfct coreelaiton and coveraince. 
    * P3 takes a simplisitc unoforim bet, across all components. The diversification proves to drop risk significantly.
  * In periods of earnings expansion and expansionary monetary policy, the portfolio will  perform well, due to long to the equities market, and short bond.   
  * Skew reduced in comparison to the other 2 portoflios. 



